In [25]:
import numpy as np
import sympy as sp
from PIL import Image
from scipy import interpolate
import matplotlib.pyplot as plt
import time

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

# Tarea 3:Interpolación Bicúbica

## Instrucciones

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el viernes 24 de Julio a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).


## Introducción

En la siguiente tarea estudiaremos un método de interpolación denominado **Interpolación Bicúbica**, utilizada frecuentemente sobre imágenes. Aplicaremos el método para aumentar la resolución de una imagen intentando preservar las propiedades de la versión original.

## Contexto

Supongamos que usted conoce $f$ y las derivadas $f_x$, $f_y$ y $f_{xy}$ dentro de las coordenadas $(0,0),(0,1),(1,0)$ y $(1,1)$ de un cuadrado unitario. La superficie que interpola estos 4 puntos es:

$$
p(x,y) = \sum\limits_{i=0}^3 \sum_{j=0}^3 a_{ij} x^i y^j.
$$

Como se puede observar el problema de interpolación se resume en determinar los 16 coeficientes $a_{ij}$ y para esto se genera un total de $16$ ecuaciones utilizando los valores conocidos de $f$,$f_x$,$f_y$ y $f_{xy}$. Por ejemplo, las primeras $4$ ecuaciones son:

$$
\begin{aligned}
f(0,0)&=p(0,0)=a_{00},\\
f(1,0)&=p(1,0)=a_{00}+a_{10}+a_{20}+a_{30},\\
f(0,1)&=p(0,1)=a_{00}+a_{01}+a_{02}+a_{03},\\
f(1,1)&=p(1,1)=\textstyle \sum \limits _{i=0}^{3}\sum \limits _{j=0}^{3}a_{ij}.
\end{aligned}
$$

Para las $12$ ecuaciones restantes se debe utilizar:

$$
\begin{aligned}
f_{x}(x,y)&=p_{x}(x,y)=\textstyle \sum \limits _{i=1}^{3}\sum \limits _{j=0}^{3}a_{ij}ix^{i-1}y^{j},\\
f_{y}(x,y)&=p_{y}(x,y)=\textstyle \sum \limits _{i=0}^{3}\sum \limits _{j=1}^{3}a_{ij}x^{i}jy^{j-1},\\
f_{xy}(x,y)&=p_{xy}(x,y)=\textstyle \sum \limits _{i=1}^{3}\sum \limits _{j=1}^{3}a_{ij}ix^{i-1}jy^{j-1}.
\end{aligned}
$$


Una vez planteadas las ecuaciones, los coeficientes se pueden obtener al resolver el problema $A\alpha=x$, donde $\alpha=\left[\begin{smallmatrix}a_{00}&a_{10}&a_{20}&a_{30}&a_{01}&a_{11}&a_{21}&a_{31}&a_{02}&a_{12}&a_{22}&a_{32}&a_{03}&a_{13}&a_{23}&a_{33}\end{smallmatrix}\right]^T$ y ${\displaystyle x=\left[{\begin{smallmatrix}f(0,0)&f(1,0)&f(0,1)&f(1,1)&f_{x}(0,0)&f_{x}(1,0)&f_{x}(0,1)&f_{x}(1,1)&f_{y}(0,0)&f_{y}(1,0)&f_{y}(0,1)&f_{y}(1,1)&f_{xy}(0,0)&f_{xy}(1,0)&f_{xy}(0,1)&f_{xy}(1,1)\end{smallmatrix}}\right]^{T}}$.


En un contexto más aplicado, podemos hacer uso de la interpolación bicúbica para aumentar la resolución de una imagen. Supongamos que tenemos la siguiente imagen de tamaño $5 \times 5$:

<img src="img1.png" width="20%"/>

Podemos ir tomando segmentos de la imagen de tamaño $2 \times 2$ de la siguiente forma:

<img src="img2.png" width="50%"/>

Por cada segmento podemos generar una superficie interpoladora mediante el algoritmo de interpolación cubica. Para el ejemplo anterior estariamos generando $16$ superficies interpoladoras distintas. La idea es hacer uso de estas superficies para estimar los valores de los pixeles correspondienets a una imagen más grande. Por ejemplo, la imagen $5 \times 5$ la podemos convertir a una imagen de $9 \times 9$ agregando un pixel entre cada par de pixeles originales además de uno en el centro para que no quede un hueco.

<img src="img3.png" width="50%"/>

Aca los pixeles verdes son los mismos que la imagen original y los azules son obtenidos de evaluar cada superficie interpoladora. Notar que existen pixeles azules que se pueden obtener a partir de dos superficies interpoladoras distintas, en esos casos se puede promediar el valor de los pixeles o simplemente dejar uno de los dos. 

Para trabajar con la interpolación bicubica necesitamos conocer los valores de $f_x$, $f_y$ y $f_{xy}$. En el caso de las imagenes solo tenemos acceso al valor de cada pixel por lo que deberemos estimar cual es el valor de estos. Para estimar $f_x$ haremos lo siguiente:

Para estimar el valor de $f_x$ en cada pixel haremos una interpolación con los algoritmos conocidos, usando tres pixels en dirección de las filas, luego derivaremos el polinomio obtenido y finalmente evaluaremos en la posición de interes. La misma idea aplica para $f_y$ solo que ahora interpolaremos en dirección de las columnas.

<img src="img5.png" width="60%"/>

Por ejemplo si queremos obtener el valor de $f_x$ en la posición $(0,0)$ (imagen de la izquierda) entonces haremos una interpolación de Lagrange utilizando los pixeles $(0,-1),(0,0)$ y $(0,1)$. Derivaremos el polinomio interpolador y evaluaremos en $(0,0)$. Por otro lado si queremos obtener el valor de $f_y$ en la posición $(0,0)$ (imagen de la derecha) entonces interpolaremos los pixeles $(-1,0),(0,0)$ y $(1,0)$. Luego derivaremos el polinomio interpolador y evaluaremos en $(0,0)$.

Para obtener $f_{xy}$ seguiremos la idea anterior. Solo que esta vez se utilizaran los valores de $f_y$ y se interpolaran estos en dirección de las filas.

# Preguntas

In [2]:
#Codigo para abrir y visualizar imágenes
#img = Image.open('imagenes_prueba/sunset.png')
#array=np.array(img)
#imgplot = plt.imshow(array)
#plt.show()

## 1. Interpolación bicubica

### 1.1  Obtener derivadas (30 puntos)

Implemente la función `derivativeValues` que reciba como input un arreglo con valores, el método de interpolación y si es que se considera el uso de  los puntos de chebyshev . La función debe retornar un arreglo de igual dimensión con los valores de las derivadas de los puntos obtenidas

Los métodos de interpolación serán representados por los siguientes valores

* Interpolación de lagrange: `'lagrange'`
* Diferencias divididas de Newton: `'newton'`
* Spline cubica: `'spline3'`


In [3]:
def newtonDD(x_i, y_i):
    n = x_i.shape[-1]
    pyramid = np.zeros((n, n)) # Create a square matrix to hold pyramid
    pyramid[:,0] = y_i # first column is y
    for j in range(1,n):
        for i in range(n-j):
            # create pyramid by updating other columns
            pyramid[i][j] = (pyramid[i+1][j-1] - pyramid[i][j-1]) / (x_i[i+j] - x_i[i])
    a = pyramid[0] # f[ ... ] coefficients
    return np.poly1d(a[::-1])

def chebyshevNodes(n):
    i = np.arange(1, n+1)
    t = (2*i - 1) * np.pi / (2 * n)
    return np.cos(t)

In [8]:
def derivativeValues(values, method, cheb):
    """
    Parameters
    ----------
    values: (int array) points values
    method: (string)    interpolation method
    cheb:   (boolean)   if chebyshev points are used

    Returns
    -------
    d: (float array) derivative value of interpolated points

    """
    values = np.sort(values)
    N = len(values)
    d = np.array(N)
    
    if cheb:
        xs = np.sort(chebyshevNodes(N))
    else:
        xs = np.linspace(0, N-1, N)
    
    if method == 'lagrange':
        f = interpolate.lagrange(xs, values)
        fp = f.deriv(1)
    elif method == 'newton':
        f = newtonDD(xs, values)
        fp = f.deriv(1)
    else:
        f = interpolate.CubicSpline(xs, values)
        fp = f.derivative()
    d = fp(xs)
    
    return d



### 1.2 Interpolación de imagen (50 puntos)
Implemente la función `bicubicInterpolation` que reciba como input la matriz de la imagen y cuantos píxeles extra se quiere agregar entre los píxeles originales y el algoritmo de interpolación a utilizar. La función debe retornar la matriz con la imagen de dimensión nueva. Considere que se debe aplicar el método de interpolación en cada canal RGB por separado.

In [37]:
def bicubicInterpolation(image, interiorPixels, method,cheb):
    """
    Parameters
    ----------
    image: (nxnx3 array) image array in RGB format
    interio Pixels: (int) interpolation method
    method: (string)    interpolation method
    cheb:   (boolean)   if chebyshev points are used


    Returns
    -------
    newImage:  (nxnx3 array) image array in RGB format

    """
    A = np.array([
        [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],
        [1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0],
        [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,1,2,3,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0],
        [0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3],
        [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0],
        [0,0,0,0,1,0,0,0,2,0,0,0,3,0,0,0],
        [0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3],
        [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,1,2,3,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,1,0,0,0,2,0,0,0,3,0,0],
        [0,0,0,0,0,1,2,3,0,2,4,6,0,3,6,9]
    ])
    
    A_i = np.linalg.inv(A)
    
    n = image.shape[0]
    image = np.append(image, image[:,-1:], axis=1)
    image = np.append(image, image[:,-1:], axis=1)
    image = np.append(image, image[:,-1:], axis=1)
    image = np.append(image, image[:,-1:], axis=1)
    rep = np.ones(image.shape[0])
    rep[-1] = 5
    image = np.repeat(image, repeats = rep.astype(int), axis=0)
    extra_p = interiorPixels(n-1) if n%2 else interiorPixels*n
    newImage = np.zeros((n+extra_p+1, n+extra_p+1, 3))
    added_p = 1
    i_n = 0
    for i in range(n+1):
        j_n = 0
        for j in range(n+1):
            f_00 = image[i ][j  ]
            f_01 = image[i ][j+1]
            f_10 = image[i+1][j  ]
            f_11 = image[i+1][j+1]

            fxR_00  = derivativeValues(np.array([image[i][j][0], image[i+1][j][0], image[i+2][j][0]]), method, cheb)
            fxG_00  = derivativeValues(np.array([image[i][j][1], image[i+1][j][1], image[i+2][j][1]]), method, cheb)
            fxB_00  = derivativeValues(np.array([image[i][j][2], image[i+1][j][2], image[i+2][j][2]]), method, cheb)

            fxR_01  = derivativeValues(np.array([image[i][j+1][0], image[i+1][j+1][0], image[i+2][j+1][0]]), method, cheb)
            fxG_01  = derivativeValues(np.array([image[i][j+1][1], image[i+1][j+1][1], image[i+2][j+1][1]]), method, cheb)
            fxB_01  = derivativeValues(np.array([image[i][j+1][2], image[i+1][j+1][2], image[i+2][j+1][2]]), method, cheb)

            fxR_10  = derivativeValues(np.array([image[i+1][j][0], image[i+2][j][0], image[i+3][j][0]]), method, cheb)
            fxG_10  = derivativeValues(np.array([image[i+1][j][1], image[i+2][j][1], image[i+3][j][1]]), method, cheb)
            fxB_10  = derivativeValues(np.array([image[i+1][j][2], image[i+2][j][2], image[i+3][j][2]]), method, cheb)

            fxR_11  = derivativeValues(np.array([image[i+1][j+1][0], image[i+2][j+1][0], image[i+3][j+1][0]]), method, cheb)
            fxG_11  = derivativeValues(np.array([image[i+1][j+1][1], image[i+2][j+1][1], image[i+3][j+1][1]]), method, cheb)
            fxB_11  = derivativeValues(np.array([image[i+1][j+1][2], image[i+2][j+1][2], image[i+3][j+1][2]]), method, cheb)

            fyR_00  = derivativeValues(np.array([image[i][j+1][0], image[i][j+1][0], image[i][j+2][0]]), method, cheb)
            fyG_00  = derivativeValues(np.array([image[i][j+1][1], image[i][j+1][1], image[i][j+2][1]]), method, cheb)
            fyB_00  = derivativeValues(np.array([image[i][j+1][2], image[i][j+1][2], image[i][j+2][2]]), method, cheb)

            fyR_01  = derivativeValues(np.array([image[i][j+1][0], image[i][j+2][0], image[i][j+3][0]]), method, cheb)
            fyG_01  = derivativeValues(np.array([image[i][j+1][1], image[i][j+2][1], image[i][j+3][1]]), method, cheb)
            fyB_01  = derivativeValues(np.array([image[i][j+1][2], image[i][j+2][2], image[i][j+3][2]]), method, cheb)

            fyR_10  = derivativeValues(np.array([image[i+1][j][0], image[i+2][j][0], image[i+3][j][0]]), method, cheb)
            fyG_10  = derivativeValues(np.array([image[i+1][j][1], image[i+2][j][1], image[i+3][j][1]]), method, cheb)
            fyB_10  = derivativeValues(np.array([image[i+1][j][2], image[i+2][j][2], image[i+3][j][2]]), method, cheb)

            fyR_11  = derivativeValues(np.array([image[i+1][j+1][0], image[i+2][j+1][0], image[i+3][j+1][0]]), method, cheb)
            fyG_11  = derivativeValues(np.array([image[i+1][j+1][1], image[i+2][j+1][1], image[i+3][j+1][1]]), method, cheb)
            fyB_11  = derivativeValues(np.array([image[i+1][j+1][2], image[i+2][j+1][2], image[i+3][j+1][2]]), method, cheb)

            fxyR_00 = derivativeValues(fyR_00, method, cheb)
            fxyG_00 = derivativeValues(fyG_00, method, cheb)
            fxyB_00 = derivativeValues(fyB_00, method, cheb)

            fxyR_01 = derivativeValues(fyR_01, method, cheb)
            fxyG_01 = derivativeValues(fyG_01, method, cheb)
            fxyB_01 = derivativeValues(fyB_01, method, cheb)

            fxyR_10 = derivativeValues(fyR_10, method, cheb)
            fxyG_10 = derivativeValues(fyG_10, method, cheb)
            fxyB_10 = derivativeValues(fyB_10, method, cheb)

            fxyR_11 = derivativeValues(fyR_11, method, cheb)
            fxyG_11 = derivativeValues(fyG_11, method, cheb)
            fxyB_11 = derivativeValues(fyB_11, method, cheb)


            fsR = np.array([f_00[0], f_10[0], f_01[0], f_11[0], fxR_00[0], fxR_10[0], fxR_01[0], fxR_11[0], fyR_00[0],
                            fyR_10[0], fyR_01[0], fyR_11[0], fxyR_00[0], fxyR_10[0], fxyR_01[0], fxyR_11[0]])

            fsG = np.array([f_00[1], f_10[1], f_01[1], f_11[1], fxG_00[0], fxG_10[0], fxG_01[0], fxG_11[0], fyG_00[0],
                            fyG_10[0], fyG_01[0], fyG_11[0], fxyG_00[0], fxyG_10[0], fxyG_01[0], fxyG_11[0]])

            fsB = np.array([f_00[2], f_10[2], f_01[2], f_11[2], fxB_00[0], fxB_10[0], fxB_01[0], fxB_11[0], fyB_00[0],
                            fyB_10[0], fyB_01[0], fyB_11[0], fxyB_00[0], fxyB_10[0], fxyB_01[0], fxyB_11[0]])

            alpha_R = np.dot(A_i,fsR).reshape((4,4)).T
            alpha_G = np.dot(A_i,fsG).reshape((4,4)).T
            alpha_B = np.dot(A_i,fsB).reshape((4,4)).T

            f_aux = lambda x: np.array([1, x, x**2, x**3])

            pR_00 = np.dot(np.dot(f_aux(0),alpha_R),f_aux(0))
            pG_00 = np.dot(np.dot(f_aux(0),alpha_G),f_aux(0))
            pB_00 = np.dot(np.dot(f_aux(0),alpha_B),f_aux(0))

            pR_01 = np.dot(np.dot(f_aux(0),alpha_R),f_aux(1))
            pG_01 = np.dot(np.dot(f_aux(0),alpha_G),f_aux(1))
            pB_01 = np.dot(np.dot(f_aux(0),alpha_B),f_aux(1))

            pR_10 = np.dot(np.dot(f_aux(1),alpha_R),f_aux(0))
            pG_10 = np.dot(np.dot(f_aux(1),alpha_G),f_aux(0))
            pB_10 = np.dot(np.dot(f_aux(1),alpha_B),f_aux(0))

            pR_11 = np.dot(np.dot(f_aux(1),alpha_R),f_aux(1))
            pG_11 = np.dot(np.dot(f_aux(1),alpha_G),f_aux(1))
            pB_11 = np.dot(np.dot(f_aux(1),alpha_B),f_aux(1))

            if j_n < n+extra_p and i_n < n+extra_p:
                newImage[i_n  ][j_n  ] = [pR_00, pG_00, pB_00]
                newImage[i_n  ][j_n+1] = [pR_01, pG_01, pB_01]
                newImage[i_n+1][j_n  ] = [pR_10, pG_10, pB_10]
                newImage[i_n+1][j_n+1] = [pR_11, pG_11, pB_11]
            j_n += 2
        i_n += 2
        
    return newImage[:n+extra_p-1, :n+extra_p-1]

## 2. Evaluacion de algoritmos



### 2.1 Tiempo de ejecucion 
Implemente la funcion `timeInterpolation` que mida el tiempo de interpolacion de una imagen dado el algoritmo de interpolacion , en segundos.(5 puntos)

In [24]:
def timeInterpolation(image, interiorPixels, method,cheb):
    """
    Parameters
    ----------
    image:	(nxnx3 array) image array in RGB format
    interiorPixels:	(int)	interpolation method
    method:	(string)	interpolation method
    cheb:	(boolean)	if chebyshev points are used


    Returns
    -------
    time:	(float) time in seconds

    """
    start_t = time.time()
    interpolated_img = bicubicInterpolation(image, interiorPixels, method,cheb)
    end_t = time.time()
    return end_t-start_t

***Pregunta: ¿Cual es el metodo que presenta mayor velocidad en general? (5 puntos)***

Diferencias divididas de Newton es el método más veloz de los tres utilizados.

Si ordenamos los métodos por su velocidad vemos que:
$$
    Newton > CubicSpline > Lagrange
$$

También se observa que, si utilizamos nodos de Chevyshev en vez de puntos equiespaciados, el tiempo de ejecución se ve mejorado.


### 2.2 Calculo de error
Implemente la funcion `errorInterpolation` la cual debe obtener el error de la imagen obtenida comparandola con una de referencia. El error debe ser calculado utilizando el indice SSIM (Structural similarity) (5 puntos)

In [43]:
def errorInterpolation(original,new):
    """
    Parameters
    ----------
    image:	(nxn array) original image array in RGB format
    new:	(nxn array) new image array in RGB format obtained from interpolation


    Returns
    -------
    error:	(float) difference between images 

    """
    original = img_as_float(original)
    new = img_as_float(original)
    error = ssim(original, new, data_range=original.max() - original.min(),multichannel=True)
    return error

***Pregunta: ¿Cual metodo presenta menor error? (5 puntos)***

# Consideraciones

* Solo trabajaremos con imagenes cuadradas
* En el caso que el valor interpolado de un punto sea mayor a 255 o menor a 0, este se trunca a 255 o 0 respectivamente
* Esta permitido el uso de sympy para calcular derivadas y para construir el polinomio interpolador 
* El calculo de error puede ser calculado utilizando la imagen en escala de grises [(ejemplo)](https://scikit-image.org/docs/dev/auto_examples/transform/plot_ssim.html)

# Referencias

* Función que genera Nodos de Chevyshev fue obtenida del material del curso.
* Funcion para las diferencias divididas se obtuvo su base del material del curso, luego se le modificaron las últimas lines.

[Enlace del jupyter que se uso como fuente](https://github.com/sct-utfsm/INF-285/blob/master/material/05_interpolacion_1D/interpolacion.ipynb)

